In [ ]:
import findspark
findspark.init('D:\spark\spark-3.3.2-bin-hadoop2')

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [ ]:
mongo_ip = 'mongodb://localhost:27017/Cleaned_Insurance'

In [ ]:
spark = SparkSession \
.builder\
.appName("myApp1")\
.config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1")\
.getOrCreate()

In [ ]:
ben_8 = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri",mongo_ip + ".ben_2008").load()

In [ ]:
ben_9 = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri",mongo_ip + ".ben_2009").load()

In [ ]:
ben_10 = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri",mongo_ip + ".ben_2010").load()

In [ ]:
Inp =  spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri",mongo_ip + ".Inpatient").load()

In [ ]:
ben_10.dtypes

In [ ]:
from pyspark.ml.feature import Bucketizer

boundaries = [25,30,40,50,60,70,80,90,100]
bucketizer = Bucketizer(splits=boundaries, inputCol="Age", outputCol="Age_Group")
ben_10 = bucketizer.transform(ben_10)

In [ ]:
df.dtypes

In [ ]:
df.createOrReplaceTempView("df")

In [ ]:
spark.sql("select Age,Age_Group from ben_8 where Age>90").show()

In [ ]:
df.select('Gender').distinct().collect()

## Mortality Rate By Gender


In [ ]:
from pyspark.sql.functions import when
df = df.withColumn("Gender", when(df["Gender"] == 1 , 'M').otherwise('F'))

In [ ]:
from pyspark.sql.functions import when

In [ ]:
df.createOrReplaceTempView("df")

In [ ]:
mr = spark.sql('select Gender,count(*) as Mortality_count_by_Gender from df where Date_of_death is not null group by Gender')

In [ ]:
MR = mr.toPandas()

In [ ]:
MR.to_csv('C:/Users/vishw/OneDrive/Desktop/Insurance/Streamlit/BENEF/B_9/b_9_Mortality_Rate_By_Gender.csv',  index=False)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.barplot(x = 'Gender',y = 'Mortality_count_by_Gender',data = MR)
plt.show()

## Mortality Rate By AgeGroup

In [ ]:
from pyspark.ml.feature import Bucketizer

boundaries = [25,30,40,50,60,70,80,90,100]
bucketizer = Bucketizer(splits=boundaries, inputCol="Age", outputCol="Age_Group")
ben_10 = bucketizer.transform(ben_10)

In [ ]:
from pyspark.sql.functions import when
ben_10 = ben_10.withColumn("Age_Group", when(ben_10.Age_Group == 0.0, "25-30").when(ben_10.Age_Group == 1.0, "31-40").when(ben_10.Age_Group == 2.0, "41-50").when(ben_10.Age_Group == 3.0, "51-60").when(ben_10.Age_Group == 4.0, "61-70").when(ben_10.Age_Group == 5.0, "71-100").when(ben_10.Age_Group == 6.0, "101-90").otherwise("91-100"))

In [ ]:
ben_10.createOrReplaceTempView("ben_10")

In [ ]:
ben_10.columns

In [ ]:
b_10 = spark.sql("select Age_Group,count(Date_of_death) as cnt from ben_10 where Date_of_death is not null group by Age_Group order by cnt desc")
b_10_pandas = b_10.toPandas()

In [ ]:
b_10.show()

In [ ]:
b_10_pandas.to_csv('C:/Users/vishw/OneDrive/Desktop/Insurance/Streamlit/BENEF/B_10/b_10_Mortality_Rate_By_AgeGroup.csv',index=False)

## Mortality rate Disease Wise per year

### 2008

In [ ]:
from pyspark.sql.functions import col
bmr_8 = ben_8.select(col("DESYNPUF_ID"),col('Date_of_death'))
Inr = Inp.select(col('DESYNPUF_ID'),col('Claim_Admitting_Diagnosis_Code'))
joined_df = bmr_8.join(Inr, on="DESYNPUF_ID", how="inner")

In [ ]:
joined_df.createOrReplaceTempView('joined')

In [ ]:
spark.sql('select Claim_Admitting_Diagnosis_Code,count(Date_of_death) as cnt \
from joined group by Claim_Admitting_Diagnosis_Code order by cnt desc limit 10').show()

In [ ]:
code.createOrReplaceTempView("code")

In [ ]:
code_p = code.toPandas()

In [ ]:
code_p.to_csv('C:/Users/vishw/OneDrive/Desktop/Insurance/Streamlit/BENEF/B_8/b_8_Mortality_Rate_Di_By_Gender.csv',index=False)

In [ ]:
count_pandas = count.toPandas()

In [ ]:
# saving the dataframe
count_pandas.to_csv('C:/Users/vishw/OneDrive/Desktop/Insurance/Streamlit/file.csv',  index=False)


### 2009

In [ ]:
from pyspark.sql.functions import col
bmr_9 = ben_9.select(col("DESYNPUF_ID"),col('Date_of_death'))
Inr = Inp.select(col('DESYNPUF_ID'),col('Claim_Admitting_Diagnosis_Code'))
joined_df = bmr_9.join(Inr, on="DESYNPUF_ID", how="inner")

joined_df.createOrReplaceTempView('joined')

a = spark.sql('select Claim_Admitting_Diagnosis_Code,count(Date_of_death) as cnt from joined group by Claim_Admitting_Diagnosis_Code order by cnt desc limit 10')

In [ ]:
a_p = a.toPandas()

In [ ]:
a_p.to_csv('C:/Users/vishw/OneDrive/Desktop/Insurance/Streamlit/BENEF/B_9/b_9_Mortality_Rate_Di_By_Gender.csv',index=False)

### 2010

In [ ]:
from pyspark.sql.functions import col
bmr_10 = ben_10.select(col("DESYNPUF_ID"),col('Date_of_death'))
Inr = Inp.select(col('DESYNPUF_ID'),col('Claim_Admitting_Diagnosis_Code'))
joined_df = bmr_10.join(Inr, on="DESYNPUF_ID", how="inner")

joined_df.createOrReplaceTempView('joined')

c= spark.sql('select Claim_Admitting_Diagnosis_Code,count(Date_of_death) as cnt from joined group by Claim_Admitting_Diagnosis_Code order by cnt desc limit 10')

In [ ]:
c_p = c.toPandas()

In [ ]:
c_p.to_csv('C:/Users/vishw/OneDrive/Desktop/Insurance/Streamlit/BENEF/B_10/b_10_Mortality_Rate_Di_By_Gender.csv',index=False)

In [ ]:
ben_10.columns

## Maximum claim on which Disease

In [ ]:
spark.sql('select Claim_Admitting_Diagnosis_Code,count(*) cnt from inp group by Claim_Admitting_Diagnosis_Code order by cnt desc limit 10').show()

In [ ]:
ben_10.createOrReplaceTempView("ben_10")

## Max Death on which Age_Group 2009

In [ ]:
ben_10 = ben_10.withColumn("Age_Group", when(ben_10.Age_Group == 0.0, "25-30").when(ben_10.Age_Group == 1.0, "31-40").when(ben_10.Age_Group == 2.0, "41-50").when(ben_10.Age_Group == 3.0, "51-60").when(ben_10.Age_Group == 4.0, "61-70").when(ben_10.Age_Group == 5.0, "71-80").when(ben_10.Age_Group == 6.0, "81-90").otherwise("91-100"))

In [ ]:
ben_10 = spark.sql("select Age_Group,count(Date_of_death) as cnt from ben_10 where Date_of_death is not null group by Age_Group order by cnt desc")

In [ ]:
ben_8.show()

In [ ]:
ben_10.show()

In [ ]:
ben_9.show()

In [ ]:
ben_9 = ben_9.withColumn("Age_Group", when(ben_9.Age_Group == 0.0, "25-30").when(ben_9.Age_Group == 1.0, "31-40").when(ben_9.Age_Group == 2.0, "41-50").when(ben_9.Age_Group == 3.0, "51-60").when(ben_9.Age_Group == 4.0, "61-70").when(ben_9.Age_Group == 5.0, "71-80").when(ben_9.Age_Group == 6.0, "81-90").otherwise("91-100"))

In [ ]:
b_8_pandas = ben_8.toPandas()

In [ ]:
ben_10.createOrReplaceTempView("ben_10")

In [ ]:
b_8_pandas.to_csv('C:/Users/vishw/OneDrive/Desktop/Insurance/Streamlit/BENEF/B_8/b_8_Mortality_Rate_By_AgeGroup.csv',index=False)

## Max Death on which Age_Group 2010

In [ ]:
from pyspark.sql.functions import col,when
ben_10 = ben_10.withColumn("Age_Group", when(ben_10.Age_Group == 0.0, "25-30").when(ben_10.Age_Group == 1.0, "31-40").when(ben_10.Age_Group == 2.0, "41-50").when(ben_10.Age_Group == 3.0, "51-60").when(ben_10.Age_Group == 4.0, "61-70").when(ben_10.Age_Group == 5.0, "71-80").when(ben_10.Age_Group == 6.0, "81-90").otherwise("91-100"))

In [ ]:
ben_10.createOrReplaceTempView("ben_10")

In [ ]:
b_10 = spark.sql("select Age_Group,count(Date_of_death) as cnt from ben_10 where Date_of_death is not null group by Age_Group order by cnt desc")
b_10_pandas = b_10.toPandas()

In [ ]:
spark.sql("select Age_Group,count(Date_of_death) as cnt from ben_10 where Date_of_death is not null group by Age_Group order by cnt desc").show()

In [ ]:
#b_10_pandas = b_10_pandas.set_index('Age_Group')

In [ ]:
b_10_pandas.to_csv('C:/Users/vishw/OneDrive/Desktop/Insurance/Streamlit/BENEF/B_8/b_8_Mortality_Rate_By_AgeGroup.csv',index=False)

## Max_death on which Age Group 2008

In [ ]:
ben_10 = ben_10.withColumn("Age_Group", when(ben_10.Age_Group == 0.0, "25-30").when(ben_10.Age_Group == 1.0, "31-40").when(ben_10.Age_Group == 2.0, "41-50").when(ben_10.Age_Group == 3.0, "51-60").when(ben_10.Age_Group == 4.0, "61-70").when(ben_10.Age_Group == 5.0, "71-80").when(ben_10.Age_Group == 6.0, "81-90").otherwise("91-100"))
df.createOrReplaceTempView("ben_10")
b_10 = spark.sql("select Age_Group,count(Date_of_death) as cnt from ben_10 where Date_of_death is not null group by Age_Group order by cnt desc")
b_10_pandas = b_10.toPandas()

b_10_pandas = b_10_pandas.set_index('Age_Group')

b_10_pandas.to_csv('C:/Users/vishw/OneDrive/Desktop/Insurance/Streamlit/BENEF/B_8/b_8_Mortality_Rate_By_AgeGroup.csv',index=False)

## Claim Days

In [ ]:
from pyspark.sql.functions import datediff
Inp = Inp.withColumn("Claim_days", datediff(Inp.Claim_End_Date, Inp.Claim_Start_Date))

In [ ]:
Inp.createOrReplaceTempView("inp")

In [ ]:
spark.sql("select Claim_End_Date,Claim_Start_Date,Claim_days from inp").show()

## Which diease has maximum claim days

In [ ]:
spark.sql('select Claim_Admitting_Diagnosis_Code,Claim_days from inp\
where Claim_days in (select max(Claim_days) from inp)').show()

In [ ]:
spark.sql('select min(Claim_days) AS MAX,Claim_Admitting_Diagnosis_Code from inp GROUP BY Claim_Admitting_Diagnosis_Code ORDER BY MAX').show()

In [ ]:
spark.sql('select COUNT(Claim_Admitting_Diagnosis_Code) from inp where Claim_days = 35').show()

In [ ]:
spark.sql('select COUNT(Claim_Admitting_Diagnosis_Code) from inp where Claim_days = 0').show()

## Average Stay at Hospital ()

In [ ]:
spark.sql('select avg(Claim_Utilization_Day_Count) as avg_hsp_stay from inp').show()

In [ ]:
spark.stop()

In [ ]:
Total_Number_of_diseases_having_max_claim_days